In [1]:
import os
import pandas as pd
import chardet

# Função para detectar encoding de um arquivo
def detectar_encoding(caminho_arquivo):
    with open(caminho_arquivo, 'rb') as f:
        resultado = chardet.detect(f.read())
    return resultado['encoding']

# Diretório onde estão as tabelas de descrição
diretorio_tabelas = './pasta_destino'

# Carregar a tabela processos com detecção de encoding
processos_caminho = os.path.join(diretorio_tabelas, 'v_processos_CodEmpresa_92577.csv')
encoding_processos = detectar_encoding(processos_caminho)
processos = pd.read_csv(processos_caminho, delimiter=';', encoding=encoding_processos)

# Remover espaços extras nas colunas e valores
processos.columns = processos.columns.str.strip()
processos = processos.map(lambda x: x.strip() if isinstance(x, str) else x)

# Lista de tabelas de descrição com a coluna de código correspondente na tabela processos
tabelas_descricao = [
    ('v_area_atuacao_CodEmpresa_92577.csv', 'codarea_acao', 'descricao'),
    ('v_clientes_CodEmpresa_92577.csv', 'cod_cliente', 'razao_social'),
    ('v_objeto_acao_CodEmpresa_92577.csv', 'objeto_acao', 'descricao'),
    ('v_localizador_CodEmpresa_92577.csv', 'destino', 'descricao'),
    ('v_prognosticos_CodEmpresa_92577.csv', 'codprognostico', 'descricao'),
    ('v_assunto_CodEmpresa_92577.csv', 'codassunto', 'descricao'),
    ('v_local_tramite_CodEmpresa_92577.csv', 'codlocaltramite', 'descricao'),
    ('v_comarca_CodEmpresa_92577.csv', 'codcomarca', 'descricao'),
    ('v_statusprocessual_CodEmpresa_92577.csv', 'statusprocessual', 'descricao'),
    ('v_grupo_processo_CodEmpresa_92577.csv', 'grupo_processo', 'descricao'),
    ('v_fase_CodEmpresa_92577.csv', 'codigo_fase', 'fase'),
]

# Atualizar a tabela processos
for tabela, processos_code_col, descricao_col in tabelas_descricao:
    # Construir o caminho completo para o arquivo CSV de descrição
    caminho_tabela = os.path.join(diretorio_tabelas, tabela)
    
    # Detectar o encoding do arquivo de descrição
    encoding_descricao = detectar_encoding(caminho_tabela)
    
    # Carregar a tabela de descrição com o encoding correto
    descricao_df = pd.read_csv(caminho_tabela, delimiter=';', encoding=encoding_descricao)
    
    # Limpar os dados: remover espaços extras nas colunas e valores
    descricao_df.columns = descricao_df.columns.str.strip()
    descricao_df = descricao_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    
    # Verificar se a coluna de código existe em ambos os DataFrames
    if processos_code_col in processos.columns and 'codigo' in descricao_df.columns:
        # Realizar o merge usando left_on (coluna específica da tabela processos) e right_on ('codigo' da descrição)
        processos = processos.merge(
            descricao_df[['codigo', descricao_col]],
            left_on=processos_code_col,
            right_on='codigo',
            how='left',
            suffixes=('', '_desc')  # Evita conflito de colunas
        )
        
        # Substituir o código pela descrição
        processos[processos_code_col] = processos[descricao_col]
        
        # Remover colunas desnecessárias após o merge
        processos.drop(columns=['codigo', descricao_col], inplace=True)
        
        print(f"Atualizado com sucesso: {tabela}")
    else:
        print(f"Coluna '{processos_code_col}' ou 'codigo' não encontrada em '{tabela}'.")

# Salvar a tabela final
processos.to_csv('processos_atualizados.csv', index=False, sep=';', encoding='utf-8')



Atualizado com sucesso: v_area_atuacao_CodEmpresa_92577.csv
Atualizado com sucesso: v_clientes_CodEmpresa_92577.csv
Atualizado com sucesso: v_objeto_acao_CodEmpresa_92577.csv
Atualizado com sucesso: v_localizador_CodEmpresa_92577.csv
Atualizado com sucesso: v_prognosticos_CodEmpresa_92577.csv
Atualizado com sucesso: v_assunto_CodEmpresa_92577.csv
Atualizado com sucesso: v_local_tramite_CodEmpresa_92577.csv
Atualizado com sucesso: v_comarca_CodEmpresa_92577.csv
Atualizado com sucesso: v_statusprocessual_CodEmpresa_92577.csv
Atualizado com sucesso: v_grupo_processo_CodEmpresa_92577.csv
Atualizado com sucesso: v_fase_CodEmpresa_92577.csv


In [2]:
pd.set_option('display.max_columns', None)

# Visualizar os primeiros registros da tabela atualizada
processos.head()

,cod_empresa,numero_processo,pasta,numero_interno,data_distribuicao,codarea_acao,tipo_acao,local_tramite,data_atendimento,vinculo_tipo,cod_cliente,cod_usuario,cod_advogado_usuario,cod_parte_adversa,cod_adv_parte_adversa,valor_causa,observacoes,data_encerramento,inclusao,ativo,objeto_acao,etiqueta,destino,tipo,codorigem,data_entrada,prazo,previsao_saida,cod_processo_apensar,exibir_apenso_raiz,favorito,codprognostico,tipoprocesso,codassunto,coddetalhesadm,codobjetoadm,origem,codlocaltramite,uf,codcomarca,prognostico,statusprocessual,grupo_processo,codigo_fase,data_contratacao,pedido,codparceiros,data_transitojulgado,data_sentenca,data_execucao,contingencia,valor_causa2,migracao,acao_ajuizada,cod_fase_processo,campo_livre3,processo_traduzido,campo_livre2,campo_livre1,campo_livre4,numero_pasta,numero_cnj,migra_parceiro,unificador_cliente,cod_cliente_removido,cnj_ok,unificador_processo,cod_processo_removido,migracao2,numero_vara,migracao3,data_ultima_visualizacao,segredo_justica,capturar_andamentos,codigo_desc
0,92577,Aguardando numeração,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fulano(a) 225,347394,0,0,0,0.0,NaN,NaN,18/07/2018 11:51,2,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,S,0,NaN,J,NaN,0,0,NaN,NaN,UF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1548.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,18/07/2018 11:51,NaN,NaN,NaN
1,92577,"1,23451E+17",0.0,NaN,26/06/2018 00:00,Direito Cível,NaN,NaN,NaN,NaN,Fulano(a) 758,301340,0,0,0,0.0,NaN,NaN,14/12/2017 18:19,0,Indenização,NaN,Arquivo Morto,0,0,NaN,NaN,NaN,NaN,S,0,Procedência Parcial Provável,J,Indenização,0,253559,NaN,Juizado Especial Cível,MG,Ipatinga,NaN,Autor,Justiça Estadual,Arquivado,14/12/2017 00:00,INDENIZAÇÃO SPC,15441751,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,67701.0,NaN,1375.0,0012345-12.2024.1.12.0000,NaN,NaN,NaN,1.0,0,0,NaN,1.0,NaN,18/10/2018 15:14,NaN,NaN,1.0
2,92577,Aguardando numeração,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fulano(a) 940,347394,0,0,0,0.0,NaN,NaN,22/08/2017 17:16,2,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,S,0,NaN,J,NaN,0,0,NaN,NaN,UF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1264.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,22/08/2017 17:16,NaN,NaN,NaN
3,92577,Aguardando numeração,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fulano(a) 1327,347394,0,0,0,0.0,NaN,NaN,29/06/2017 16:11,2,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,S,0,NaN,J,NaN,0,0,NaN,NaN,UF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,1214.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,28/09/2017 14:34,NaN,NaN,NaN
4,92577,"1,23451E+17",0.0,NaN,22/10/2014 00:00,Direito Cível,NaN,NaN,NaN,NaN,Fulano(a) 1447,301340,0,0,0,15000.0,NaN,07/09/2015 00:00,14/08/2014 11:07,0,Indenização,NaN,NaN,0,0,NaN,NaN,NaN,NaN,S,0,Procedência Provável,J,Indenização,0,253559,NaN,Juizado Especial Cível,MG,Ipatinga,NaN,Autor,Justiça Estadual,Instrução,14/08/2014 00:00,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,67699.0,NaN,561.0,0012345-12.2024.1.12.0001,NaN,NaN,NaN,1.0,0,0,NaN,1.0,NaN,22/04/2019 15:26,NaN,NaN,1.0


In [8]:
# Carregar o arquivo CSV
processos = pd.read_csv('processos_atualizados.csv', delimiter=';', encoding='utf-8')

# Lista de colunas a serem removidas
colunas_para_remover = [
    'cod_empresa', 'pasta', 'numero_interno', 'tipo_acao', 'local_tramite', 
    'vinculo_tipo', 'cod_advogado_usuario', 
    'cod_adv_parte_adversa', 'ativo', 'etiqueta', 'tipo', 'codorigem', 
    'data_entrada', 'prazo', 'previsao_saida', 'cod_processo_apensar', 
    'exibir_apenso_raiz', 'favorito', 'tipoprocesso', 'coddetalhesadm', 
    'codobjetoadm', 'origem', 'prognostico', 'codparceiros', 'contingencia', 
    'valor_causa2', 'migração', 'acao_ajuizada', 'cod_fase', 'campo_livre3', 
    'processo_traduzido', 'campo_livre2', 'campo_livre1', 'campo_livre4', 
    'migra_parceiro', 'unificador_cliente', 'cod_cliente_removido', 'cnj_ok', 
    'unificador_processo', 'cod_processo_removido', 'migracao2', 'numero_vara', 
    'migracao3', 'segredo_justica', 'capturar_andamentos', 'codigo_desc'
]

# Remover as colunas especificadas
processos = processos.drop(columns=colunas_para_remover, errors='ignore')

# Salvar o arquivo atualizado
processos.to_csv('processos_atualizados2.csv', index=False, sep=';', encoding='utf-8')

print("Colunas removidas com sucesso!")


Colunas removidas com sucesso!


In [9]:
processos.head()

,numero_processo,data_distribuicao,codarea_acao,data_atendimento,cod_cliente,cod_usuario,cod_parte_adversa,valor_causa,observacoes,data_encerramento,inclusao,objeto_acao,destino,codprognostico,codassunto,codlocaltramite,uf,codcomarca,statusprocessual,grupo_processo,codigo_fase,data_contratacao,pedido,data_transitojulgado,data_sentenca,data_execucao,migracao,cod_fase_processo,numero_pasta,numero_cnj,data_ultima_visualizacao
0,Aguardando numeração,NaN,NaN,NaN,Fulano(a) 225,347394,0,0.0,NaN,NaN,18/07/2018 11:51,NaN,NaN,NaN,NaN,NaN,UF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1548.0,NaN,18/07/2018 11:51
1,"1,23451E+17",26/06/2018 00:00,Direito Cível,NaN,Fulano(a) 758,301340,0,0.0,NaN,NaN,14/12/2017 18:19,Indenização,Arquivo Morto,Procedência Parcial Provável,Indenização,Juizado Especial Cível,MG,Ipatinga,Autor,Justiça Estadual,Arquivado,14/12/2017 00:00,INDENIZAÇÃO SPC,NaN,NaN,NaN,NaN,NaN,1375.0,0012345-12.2024.1.12.0000,18/10/2018 15:14
2,Aguardando numeração,NaN,NaN,NaN,Fulano(a) 940,347394,0,0.0,NaN,NaN,22/08/2017 17:16,NaN,NaN,NaN,NaN,NaN,UF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1264.0,NaN,22/08/2017 17:16
3,Aguardando numeração,NaN,NaN,NaN,Fulano(a) 1327,347394,0,0.0,NaN,NaN,29/06/2017 16:11,NaN,NaN,NaN,NaN,NaN,UF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1214.0,NaN,28/09/2017 14:34
4,"1,23451E+17",22/10/2014 00:00,Direito Cível,NaN,Fulano(a) 1447,301340,0,15000.0,NaN,07/09/2015 00:00,14/08/2014 11:07,Indenização,NaN,Procedência Provável,Indenização,Juizado Especial Cível,MG,Ipatinga,Autor,Justiça Estadual,Instrução,14/08/2014 00:00,NaN,NaN,NaN,NaN,NaN,NaN,561.0,0012345-12.2024.1.12.0001,22/04/2019 15:26


In [ ]:
# Configurações
input_file = 'processos_atualizados2.csv'
output_dir = './output'
output_file = os.path.join(output_dir, 'processos.xlsx')

# Garantir que o diretório de saída existe
os.makedirs(output_dir, exist_ok=True)

# Carregar o arquivo de entrada
processos = pd.read_csv(input_file, delimiter=';', encoding='utf-8')

# Criar a nova tabela com as transformações especificadas
processos_novo = pd.DataFrame({
    'NOME DO CLIENTE': processos['cod_cliente'],
    'PARTE CONTRÁRIA': processos['cod_parte_adversa'],
    'TIPO DE AÇÃO': processos['objeto_acao'],
    'GRUPO DE AÇÃO': processos['codarea_acao'],
    'FASE PROCESSUAL': processos['codigo_fase'],
    'ETAPA': '',
    'NÚMERO DO PROCESSO': processos['numero_processo'],
    'PROCESSO ORIGINÁRIO': '',
    'TRIBUNAL': '',
    'VARA': processos['codlocaltramite'],
    'COMARCA': processos['codcomarca'],
    'PROTOCOLO': '',
    'EXPECTATIVA/VALOR DA CAUSA': processos['valor_causa'],
    'VALOR HONORÁRIOS': '',
    'PASTA': processos['numero_pasta'],
    'DATA CADASTRO': pd.to_datetime(processos['inclusao'], errors='coerce', dayfirst=True).dt.strftime('%d/%m/%Y'),
    'DATA FECHAMENTO': pd.to_datetime(processos['data_contratacao'], errors='coerce', dayfirst=True).dt.strftime('%d/%m/%Y'),
    'DATA TRANSITO': pd.to_datetime(processos['data_transitojulgado'], errors='coerce', dayfirst=True).dt.strftime('%d/%m/%Y'),
    'DATA ARQUIVAMENTO': pd.to_datetime(processos['data_encerramento'], errors='coerce', dayfirst=True).dt.strftime('%d/%m/%Y'),
    'DATA REQUERIMENTO': pd.to_datetime(processos['data_distribuicao'], errors='coerce', dayfirst=True).dt.strftime('%d/%m/%Y'),
    'RESPONSÁVEL': processos['cod_usuario'],
    'ANOTAÇÕES GERAIS': processos['observacoes'].fillna('') + ' ' + processos['pedido'].fillna(''),
})

# Salvar em formato Excel
processos_novo.to_excel(output_file, index=False, engine='openpyxl')

print(f"Arquivo salvo com sucesso em: {output_file}")


Arquivo salvo com sucesso em: ./output\processos.xlsx
